In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [3]:
data = pd.read_csv('gym_members_exercise_tracking.csv')

In [5]:
print(data.isnull().sum())

Age                              0
Gender                           0
Weight (kg)                      0
Height (m)                       0
Max_BPM                          0
Avg_BPM                          0
Resting_BPM                      0
Session_Duration (hours)         0
Calories_Burned                  0
Workout_Type                     0
Fat_Percentage                   0
Water_Intake (liters)            0
Workout_Frequency (days/week)    0
Experience_Level                 0
BMI                              0
dtype: int64


In [6]:
features = ['Age', 'Gender', 'Weight (kg)', 'Height (m)', 'BMI', 
            'Resting_BPM', 'Session_Duration (hours)', 'Calories_Burned',
            'Fat_Percentage', 'Experience_Level']

target = 'Workout_Type'

In [7]:
X = data[features]
y = data[target]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
numeric_features = ['Age', 'Weight (kg)', 'Height (m)', 'BMI', 
                   'Resting_BPM', 'Session_Duration (hours)', 'Calories_Burned',
                   'Fat_Percentage', 'Experience_Level']
categorical_features = ['Gender']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [11]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', RandomForestClassifier(random_state=42))])

In [13]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Weight (kg)',
                                                   'Height (m)', 'BMI',
                                                   'Resting_BPM',
                                                   'Session_Duration (hours)',
                                                   'Calories_Burned',
                                                   'Fat_Percentage',
                                                   'Experience_Level']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [14]:
y_pred = model.predict(X_test)
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.2564102564102564

Classification Report:
               precision    recall  f1-score   support

      Cardio       0.24      0.30      0.27        46
        HIIT       0.36      0.33      0.35        42
    Strength       0.20      0.26      0.22        47
        Yoga       0.28      0.17      0.21        60

    accuracy                           0.26       195
   macro avg       0.27      0.26      0.26       195
weighted avg       0.27      0.26      0.26       195



In [20]:
joblib.dump(model, 'workout_recommender.joblib')

['workout_recommender.joblib']

In [21]:
def recommend_workout(age, gender, weight, height, bmi, resting_bpm, 
                     session_duration, calories_burned, fat_percentage, 
                     experience_level):
    
    input_data = pd.DataFrame({
        'Age': [age],
        'Gender': [gender],
        'Weight (kg)': [weight],
        'Height (m)': [height],
        'BMI': [bmi],
        'Resting_BPM': [resting_bpm],
        'Session_Duration (hours)': [session_duration],
        'Calories_Burned': [calories_burned],
        'Fat_Percentage': [fat_percentage],
        'Experience_Level': [experience_level]
    })
    
    try:
        model
    except NameError:
        model = joblib.load('workout_recommender.joblib')
    
    workout = model.predict(input_data)[0]
    probabilities = model.predict_proba(input_data)[0]
    classes = model.named_steps['classifier'].classes_
    
    response = {
        'recommended_workout': workout,
        'probabilities': {cls: prob for cls, prob in zip(classes, probabilities)}
    }
    
    return response